<a href="https://colab.research.google.com/github/HromovAndrey/II_mood_4_part_2/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BC%D0%B5%D1%80%D0%B5%D0%B6%D1%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт модулів

In [9]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.nn.functional as F

Дані про морське вухо(молюск)

<div>
<img src="https://m.media-amazon.com/images/I/81-FC5TlLtL.jpg" width="500"/>
</div>

In [10]:
df = pd.read_csv('https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/abalone.csv')
df.drop(columns=['Type', 'WholeWeight', 'ShuckedWeight', 'VisceraWeight', 'ShellWeight'], inplace=True)

In [11]:
df.head()

,LongestShell,Diameter,Height,Rings
0,0.455,0.365,0.095,15
1,0.350,0.265,0.090,7
2,0.530,0.420,0.135,9
3,0.440,0.365,0.125,10
4,0.330,0.255,0.080,7


Потрібно на основі розмірів молюска спрогнозувати кількість кілець в мушлі `Rings`. Використовуйте процесор cpu

# Завдання 1
Створіть клас Dataset

In [12]:

scaler = StandardScaler()
X = df.drop(columns=['Rings'])
y = df['Rings'].values
X = scaler.fit_transform(X)

class AbaloneDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

dataset = AbaloneDataset(X, y)


# Завдання 2
Створіть DataLoader. Перевірте розміри даних в першому пакеті. Розмір пакетів оберіть самостійно

In [13]:
batch_size = 32


dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

first_batch = next(iter(dataloader))
print(first_batch[0].shape, first_batch[1].shape)


torch.Size([32, 3]) torch.Size([32])


# Завдання 3
Створіть нейромережу:
* Кількість шарів оберіть між 1-3
* Виберіть різні функції активації: Relu, LeakyRelu, Swish
* Кілкість нейронів у прихованих шарах повинна **не зростати**

In [14]:
class AbaloneNet(nn.Module):
    def __init__(self):
        super(AbaloneNet, self).__init__()
        self.fc1 = nn.Linear(3, 10)
        self.fc2 = nn.Linear(10, 5)
        self.fc3 = nn.Linear(5, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = AbaloneNet()
print(model)


AbaloneNet(
  (fc1): Linear(in_features=3, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=5, bias=True)
  (fc3): Linear(in_features=5, out_features=1, bias=True)
)


# Завдання 4
Визначте функцію втрат та оптимізатор

In [15]:
criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# Завдання 5
Натренуйте нейромережу. візміть невелику кількість епох

In [16]:
epochs = 10

for epoch in range(epochs):
    for inputs, targets in dataloader:
       outputs = model(inputs)
       loss = criterion(outputs.squeeze(), targets)
       optimizer.zero_grad()
       loss.backward()
       optimizer.step()
       print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")


Epoch 1/10, Loss: 101.0228
Epoch 1/10, Loss: 130.1093
Epoch 1/10, Loss: 89.2327
Epoch 1/10, Loss: 125.5445
Epoch 1/10, Loss: 98.9707
Epoch 1/10, Loss: 136.2963
Epoch 1/10, Loss: 118.1621
Epoch 1/10, Loss: 99.4574
Epoch 1/10, Loss: 113.8290
Epoch 1/10, Loss: 129.0815
Epoch 1/10, Loss: 111.5290
Epoch 1/10, Loss: 90.6939
Epoch 1/10, Loss: 121.6684
Epoch 1/10, Loss: 120.6711
Epoch 1/10, Loss: 117.4135
Epoch 1/10, Loss: 107.8164
Epoch 1/10, Loss: 110.5531
Epoch 1/10, Loss: 87.4460
Epoch 1/10, Loss: 120.3267
Epoch 1/10, Loss: 89.6261
Epoch 1/10, Loss: 126.0698
Epoch 1/10, Loss: 111.4677
Epoch 1/10, Loss: 106.9813
Epoch 1/10, Loss: 128.6653
Epoch 1/10, Loss: 101.0789
Epoch 1/10, Loss: 147.3245
Epoch 1/10, Loss: 108.7723
Epoch 1/10, Loss: 119.2879
Epoch 1/10, Loss: 95.4662
Epoch 1/10, Loss: 125.9487
Epoch 1/10, Loss: 121.1020
Epoch 1/10, Loss: 122.9319
Epoch 1/10, Loss: 103.7907
Epoch 1/10, Loss: 126.8542
Epoch 1/10, Loss: 102.3528
Epoch 1/10, Loss: 94.6047
Epoch 1/10, Loss: 108.8739
Epoch 1/1

# Завдання 6
Збережіть нейромережу та зробіть прогноз

In [17]:
torch.save(model.state_dict(), 'abalone_model.pth')

model.eval()
with torch.no_grad():
    sample = torch.tensor([0.5, 0.4, 0.1], dtype=torch.float32)
    prediction = model(sample)
    print(f"Прогнозована кількість кілець: {prediction.item():.2f}")


Прогнозована кількість кілець: 10.59
